# Kaggle contest 

![](https://storage.googleapis.com/kaggle-datasets-images/134715/320111/9230bbbadb805f8018c4dc289b72dd8c/dataset-cover.png?t=2019-05-24-07-03-05)

https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

# Download Glove (Global Vectors for Word Representation)
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip
!ls -l
!head -n 10 glove.6B.100d.txt

import nltk
nltk.download("book")

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  #gpu`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
--2020-06-14 07:51:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  2.17MB/s    in 6m 29s  

2020-06-14 07:58:28 (2.11 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
total 3039136
-rw-rw-r-- 1 root root  347116733 Aug  4  2014 glove.6B.100d.txt
-rw-rw-r-- 1 root root  693432828 Aug  4  2014 glove.6B.200

True

In [2]:
from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import re
import itertools

data = pd.read_csv('/gdrive/My Drive/sourcecode/vtc_ai/raw/12 imdb_dataset.csv')

print(data.head())
print(data.shape)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
(50000, 2)


### Clean data

Raw data:

```
Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.
```

The raw data contains invaluable characters/tokens such as dot, question mark,  HTML tag, etc... We should clean this first

Cleaned data should look like this

```
Basically there s a family where a little boy Jake thinks there s a zombie in his closet his parents are fighting all the time br br This movie is slower than a soap opera and suddenly Jake decides to become Rambo and kill the zombie br br OK first of all when you re going to make a film you must Decide if its a thriller or a drama As a drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film I expected to see a BOOGEYMAN similar movie and instead i watched a drama with some meaningless thriller spots br br out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them 
```



In [ ]:
def clean(review):
    r = review.lower()
    # remove html tags
    r = re.sub('<.*?>', '', r)
    # replace non-word characters with spaces
    r = re.sub('(\W|\d)+', ' ', r)
    return r

data['review_cleaned'] = data.review.apply(clean)

In [4]:
data.head(10)

,review,sentiment,review_cleaned
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there s a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei s love in the time of money is a...
5,"Probably my all-time favorite movie, a story o...",positive,probably my all time favorite movie a story of...
6,I sure would like to see a resurrection of a u...,positive,i sure would like to see a resurrection of a u...
7,"This show was an amazing, fresh & innovative i...",negative,this show was an amazing fresh innovative idea...
8,Encouraged by the positive comments about this...,negative,encouraged by the positive comments about this...
9,If you like original gut wrenching laughter yo...,positive,if you like original gut wrenching laughter yo...


In [ ]:
vocabulary_size = 10000
unknown_token = "UNKNOWN_TOKEN"
# sentence_start_token = "SENTENCE_START"
# sentence_end_token = "SENTENCE_END"

In [6]:
nltk.word_tokenize(data.review_cleaned.iloc[0])

['one',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching',
 'just',
 'oz',
 'episode',
 'you',
 'll',
 'be',
 'hooked',
 'they',
 'are',
 'right',
 'as',
 'this',
 'is',
 'exactly',
 'what',
 'happened',
 'with',
 'me',
 'the',
 'first',
 'thing',
 'that',
 'struck',
 'me',
 'about',
 'oz',
 'was',
 'its',
 'brutality',
 'and',
 'unflinching',
 'scenes',
 'of',
 'violence',
 'which',
 'set',
 'in',
 'right',
 'from',
 'the',
 'word',
 'go',
 'trust',
 'me',
 'this',
 'is',
 'not',
 'a',
 'show',
 'for',
 'the',
 'faint',
 'hearted',
 'or',
 'timid',
 'this',
 'show',
 'pulls',
 'no',
 'punches',
 'with',
 'regards',
 'to',
 'drugs',
 'sex',
 'or',
 'violence',
 'its',
 'is',
 'hardcore',
 'in',
 'the',
 'classic',
 'use',
 'of',
 'the',
 'word',
 'it',
 'is',
 'called',
 'oz',
 'as',
 'that',
 'is',
 'the',
 'nickname',
 'given',
 'to',
 'the',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'it',
 'focuses',
 'mainly',
 'on',
 '

### Tokenize data
You should use the library NLTK instead of simple string splitting


In [ ]:
tokenized_reviews = [nltk.word_tokenize(sent) for sent in data.review_cleaned]

### Build your own dictionary
Get a list of the n highest frequency tokens from tokenized data. Which value n should be? It depends on your experience, language knowledge, task requirements, token frequency distribution, etc...

In [9]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_reviews))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 102084 unique words tokens.


In [10]:
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))


Using vocabulary size 10000.
The least frequent word in our vocabulary is 'inheritance' and appeared 59 times.


In [15]:
word_to_index['the']

0

### Refine tokenized data:

Replace all token which does not exist in the above dictionary by special token: `UNKNOWN_TOKEN`. Remember to add this special token to our dictionary

In [ ]:
for i, sent in enumerate(tokenized_reviews):
    tokenized_reviews[i] = [w if w in word_to_index else unknown_token for w in sent]

In [17]:
print("\nExample sentence: '%s'" % data.review_cleaned.iloc[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_reviews[0])


Example sentence: 'one of the other reviewers has mentioned that after watching just oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t d

### Encode tokenzied data

Replace all tokens by their indices in the above dictionary. Each token will be encoded to one-hot-coding later based on its dictionary index

In [18]:
total_len = sum(len(r) for r in tokenized_reviews)
print(total_len / len(tokenized_reviews))

234.14006


In [19]:
encoded_reviews = [[word_to_index[word] for word in review] for review in tokenized_reviews]
print(tokenized_reviews[0])
print(encoded_reviews[0])

['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'UNKNOWN_TOKEN', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'UNKNOWN_TOKEN', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'UNKNOWN_TOKEN', 'given', 'to', 'the', 'UNKNOWN_TOKEN', 'maximum', 'security', 'state', 'UNKNOWN_TOKEN', 'it', 'focuses', 'mainly', 'on', 'UNKNOWN_TOKEN', 'city', 'an', 'experimental', 'section', 'of', 'the', 'prison', 'whe

In [ ]:
max_length = 230
encoded_reviews = [review[: max_length] for review in encoded_reviews]
label = [1 if s=='positive' else 0 for s in data.sentiment]

### Padding

Each data point here is a review with varying sizes. So, we have to choose the max length m and:
- Trim all reviews that contains more than m tokens
- Add UNKNOWN_TOKEN to the end of all reviews that contains less than m tokens


In [ ]:
pad_word = word_to_index[unknown_token]
for review in encoded_reviews:
    if len(review) < max_length:
        delta = max_length - len(review)
        padding = [pad_word] * delta
        review += padding

In [25]:
unknown_token

'UNKNOWN_TOKEN'

In [26]:
len(encoded_reviews[4])

230

In [ ]:
import numpy as np
X = np.array(encoded_reviews)
y = np.array(label)

In [28]:
X[2]

array([   8,  190,    9,   12,    2,  389,   94,    4, 1134,   56,   21,
          2,   96,  853, 1450, 2572, 1220,    7,    0,  882, 9999,  754,
          1,  148,    2,  631, 2310,  199,    0,  111,    5, 4057,   17,
          0,  407,    5, 1888,    1,    0,  100,   24, 1466,   58,    0,
         68, 6002, 6596, 1537,  469,  135,   48,  196,   28,  665,   50,
         31,  942,    9,    5,   22, 1010,  216, 2894, 5099,    8,  190,
          6,   12, 3018,   10, 2811, 1764,    5,  130, 1384,    7, 1113,
          3,    0,  392,  106,    3,  177,   26, 2038,    4,  109,    9,
         12,    0,   88,    8,  219, 1425,   30,   27,    3, 2811,   11,
       1289,    7,  154, 2994,    8,  131,    2, 2039,  135,    8,  136,
        110,   75, 1486,   16, 8117, 9999,    7,    9,   53, 1307,    4,
       1237,  175,   41, 1229, 1404,    1, 5029,  202,   82,    2,  828,
         17, 3515,  186,  244,    9,  196,   22,   28,    0, 7122, 5051,
          3,   25,  612,   17,    6,   12, 9999,   

### Split train & test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [30]:
print(X_train.shape)
print(X_test.shape)

(35000, 230)
(15000, 230)


### Training with embedding layer

In [31]:
with tf.device('/GPU:0'):
    model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocabulary_size, 300, input_length=max_length),
                tf.keras.layers.LSTM(100),

                tf.keras.layers.Flatten(),

                tf.keras.layers.Dense(100, activation='relu'),
                tf.keras.layers.Dense(2, activation='softmax')
    ])

model.compile(loss='sparse_categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15, batch_size=128)

Epoch 1/15
274/274 [==============================] - 17s 63ms/step - loss: 0.6632 - accuracy: 0.5905
Epoch 2/15
274/274 [==============================] - 17s 64ms/step - loss: 0.6476 - accuracy: 0.6241
Epoch 3/15
274/274 [==============================] - 18s 65ms/step - loss: 0.6507 - accuracy: 0.6102
Epoch 4/15
274/274 [==============================] - 17s 63ms/step - loss: 0.5521 - accuracy: 0.7119
Epoch 5/15
274/274 [==============================] - 17s 63ms/step - loss: 0.3561 - accuracy: 0.8548
Epoch 6/15
274/274 [==============================] - 17s 63ms/step - loss: 0.2593 - accuracy: 0.9001
Epoch 7/15
274/274 [==============================] - 17s 63ms/step - loss: 0.1851 - accuracy: 0.9322
Epoch 8/15
274/274 [==============================] - 17s 63ms/step - loss: 0.1267 - accuracy: 0.9569
Epoch 9/15
274/274 [==============================] - 17s 63ms/step - loss: 0.0849 - accuracy: 0.9740
Epoch 10/15
274/274 [==============================] - 17s 63ms/step - loss: 0.063

In [32]:
model.evaluate(X_test, y_test, verbose=0)

[0.576018214225769, 0.8396666646003723]

### Training with Embedding layer and LSTM

### Training with Embedding layer, LSTM, Glove

In [ ]:
with tf.device('/GPU:0'):
    model2 = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocabulary_size, 300, input_length=max_length),
                tf.keras.layers.LSTM(100),

                tf.keras.layers.Flatten(),

                tf.keras.layers.Dense(100, activation='relu'),
                tf.keras.layers.Dense(2, activation='softmax')
    ])

model2.compile(loss='sparse_categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=15, batch_size=128)